# made by https://github.com/seambr

In [ ]:
import requests
import asyncio
import aiohttp
import pandas as pd
import time
import json

In [ ]:
#@title Enter cid-list
cid_list = "28f367d8-5322-4851-8edd-9a9e8a7c3300" #@param type: "string"
print(cid_list)

28f367d8-5322-4851-8edd-9a9e8a7c3300


In [ ]:
URL = "https://www.cmnpd.org:18127/browsePage/CompoundFilter/"
r = requests.post(URL, data=json.dumps({"pageNO": 1, "pageSize": 20, "type": "compound",
                                        "cid_list": cid_list}))

cmnpd_name_list = set(r.json()["data"][0]["all_data"])
print(F"{len(cmnpd_name_list)} Compounds Found")


10 Compounds Found


In [ ]:

results = []
failed = []


def cmnpd_name_to_link(cmnpd_name):
    return f"https://www.cmnpd.org:18127/detailPage/Compound/?cid={cmnpd_name}"


async def get_url(url, cmnpd_name):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            if response.status == 200:
                cmnpd_name_list.remove(cmnpd_name)
                return await response.json()
            elif response.status == 404:
                failed.append(cmnpd_name)
                cmnpd_name_list.remove(cmnpd_name)
                return "None"

loop = True
items_to_get_at_once = 100
while loop:
    tasks = [asyncio.create_task(get_url(cmnpd_name_to_link(
        cmnpd_name), cmnpd_name)) for cmnpd_name in list(cmnpd_name_list)[:items_to_get_at_once]]
    res = await asyncio.gather(*tasks)
    for r in res:
        results.append(r)

    if (len(cmnpd_name_list) == 0):
        loop = False
    print(f"One iteration. {len(cmnpd_name_list)}")
    time.sleep(2)

print(F"{len(results)} Fetched, {len(failed)} Failed")

One iteration. 0
10 Fetched, len(failed) Failed


In [ ]:
name_list = []
iupac_name_list = []
id_list = []
for json_data in results:
    name = None
    iupac_name = None
    cmnpd_name = None
    try:
      name_data = json_data["data"]["Name_and_Classification"]
    except:
      print("Failed On This One")
      continue

    try:
        name = [i for i in name_data if i["key"] == "Name"][0]["value"]
    except:
        name = None
    try:
        iupac_name = [i for i in name_data if i["key"]
                      == "IUPAC Name"][0]["value"]
    except:
        iupac_name = None
    try:
        cmnpd_name = [i for i in name_data if i["key"]
                      == "ID"][0]["value"]
    except:
        cmnpd_name = None

    name_list.append(name)
    iupac_name_list.append(iupac_name)
    id_list.append(cmnpd_name)


In [ ]:
from google.colab import files
from pathlib import Path
name_dict = {
    "Name": name_list,
    "IUPAC_NAME": iupac_name_list
}

save_path = Path.cwd() / "name_data.csv"
df = pd.DataFrame(name_dict)
df.reset_index(drop=True)
df.to_csv(save_path,index=False,encoding="utf-8")


files.download(save_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>